In [1]:
include("../src/Atoms.jl")
include("../src/scfOptions.jl")
include("../src/anderson_mix.jl")
include("../src/kerker_mix.jl")
include("../src/Ham.jl")
include("../src/hartree_pot_bc.jl")
include("../src/pseudocharge.jl")
include("../src/getocc.jl")
include("../src/Integrators.jl")

time_evolution (generic function with 1 method)

In [2]:
dx = 0.5;
Nunit = 8;   # number of units
Lat = 10;     # size of the lattice
Ls = Nunit*Lat;
# using the default values in Lin's code
YukawaK = 0.0100
n_extra = 10;
epsil0 = 10.0;
T_elec = 100.0;

kb = 3.1668e-6;
au2K = 315774.67;
Tbeta = au2K / T_elec;

betamix = 0.5;
mixdim = 10;

Ndist  = 1;   # Temporary variable
Natoms = round(Integer, Nunit / Ndist); # number of atoms

sigma  = ones(Natoms,1)*(1.0);  # insulator
omega  = ones(Natoms,1)*0.03;
Eqdist = ones(Natoms,1)*10.0;
mass   = ones(Natoms,1)*42000.0;
nocc   = ones(Natoms,1)*2;          # number of electrons per atom
Z      = nocc;

In [3]:
function forces(x::Array{Float64,1})
    # input
    #       x: vector with the position of the atoms
    # output
    #       f: forces at the center of the atoms

    R = reshape(x, length(x), 1) # we need to make it a two dimensional array
    # creating an atom structure
    atoms = Atoms(Natoms, R, sigma,  omega,  Eqdist, mass, Z, nocc);
    # allocating a Hamiltonian
    ham = Ham(Lat, Nunit, n_extra, dx, atoms,YukawaK, epsil0, Tbeta)

    # total number of occupied orbitals
    Nocc = round(Integer, sum(atoms.nocc) / ham.nspin);

    # setting the options for the scf iteration
    mixOpts = andersonMixOptions(ham.Ns, betamix, mixdim )
    eigOpts = eigOptions(1.e-8, 1000, "eigs");
    scfOpts = scfOptions(1.e-6, 300, eigOpts, mixOpts)

    # initialize the potentials within the Hemiltonian, setting H[\rho_0]
    init_pot!(ham, Nocc)

    # running the scf iteration
    @time VtoterrHist = scf!(ham, scfOpts)

    if VtoterrHist[end] > scfOpts.SCFtol
        println("convergence not achieved!! ")
    end

    # we compute the forces 
    get_force!(ham)

    return ham.atoms.force[:]
end


forces (generic function with 1 method)

In [4]:
# Settign the time evolution

dt = 0.01

x0 = zeros(Natoms); # this is defined as an 2D array

for j = 1:Natoms
  x0[j] = (j-0.5)*Lat*Ndist+dx;
end
#x1 = x0 + dt*[1, -1 ] # with velocity
x0[1] = x0[1] + 2
x1 = x0

(x, v, vdot) = time_evolution(velocity_verlet, x -> 10*forces(x), dt, 20, x0, x1)

initial conditions
  3.394663 seconds (5.74 M allocations: 430.407 MiB, 6.44% gc time)
running the loop
  0.878244 seconds (1.13 M allocations: 210.544 MiB, 3.91% gc time)
  0.830885 seconds (1.13 M allocations: 210.377 MiB, 3.01% gc time)
  0.842353 seconds (1.13 M allocations: 209.467 MiB, 3.37% gc time)
  0.835232 seconds (1.13 M allocations: 209.663 MiB, 3.01% gc time)
  0.852453 seconds (1.13 M allocations: 209.535 MiB, 3.52% gc time)
  0.834780 seconds (1.13 M allocations: 209.754 MiB, 3.42% gc time)
  0.888504 seconds (1.13 M allocations: 209.424 MiB, 2.87% gc time)
  0.877756 seconds (1.14 M allocations: 211.197 MiB, 3.34% gc time)
  0.845026 seconds (1.13 M allocations: 209.864 MiB, 3.03% gc time)
  0.830079 seconds (1.14 M allocations: 211.245 MiB, 3.08% gc time)
  0.864344 seconds (1.13 M allocations: 210.336 MiB, 3.21% gc time)
  0.854230 seconds (1.13 M allocations: 210.047 MiB, 3.11% gc time)
  0.845546 seconds (1.13 M allocations: 209.334 MiB, 3.53% gc time)
  0.838379 s

([7.5 15.5 … 65.5 75.5; 7.5 15.5 … 65.5 75.5; … ; 7.49903 15.5009 … 65.5 75.5001; 7.49892 15.501 … 65.5 75.5001], [0.0 0.0 … 0.0 0.0; -0.000538771 0.000482747 … 2.40118e-8 5.586e-5; … ; -0.0102298 0.00916498 … 4.99806e-7 0.00106124; -0.0107674 0.00964652 … 5.31078e-7 0.00111709], [-0.0538773 0.0482748 … 2.40023e-6 0.00558601; -0.053877 0.0482745 … 2.40214e-6 0.005586; … ; -0.0537683 0.0481608 … 3.09004e-6 0.00558447; -0.0537566 0.0481485 … 3.16449e-6 0.0055843])

In [5]:
x

21×8 Array{Float64,2}:
 7.5      15.5     25.5  35.5  45.5  55.5  65.5  75.5   
 7.5      15.5     25.5  35.5  45.5  55.5  65.5  75.5   
 7.49999  15.5     25.5  35.5  45.5  55.5  65.5  75.5   
 7.49998  15.5     25.5  35.5  45.5  55.5  65.5  75.5   
 7.49996  15.5     25.5  35.5  45.5  55.5  65.5  75.5   
 7.49993  15.5001  25.5  35.5  45.5  55.5  65.5  75.5   
 7.4999   15.5001  25.5  35.5  45.5  55.5  65.5  75.5   
 7.49987  15.5001  25.5  35.5  45.5  55.5  65.5  75.5   
 7.49983  15.5002  25.5  35.5  45.5  55.5  65.5  75.5   
 7.49978  15.5002  25.5  35.5  45.5  55.5  65.5  75.5   
 7.49973  15.5002  25.5  35.5  45.5  55.5  65.5  75.5   
 7.49967  15.5003  25.5  35.5  45.5  55.5  65.5  75.5   
 7.49961  15.5003  25.5  35.5  45.5  55.5  65.5  75.5   
 7.49954  15.5004  25.5  35.5  45.5  55.5  65.5  75.5   
 7.49947  15.5005  25.5  35.5  45.5  55.5  65.5  75.5001
 7.49939  15.5005  25.5  35.5  45.5  55.5  65.5  75.5001
 7.49931  15.5006  25.5  35.5  45.5  55.5  65.5  75.5001
 7.49922

In [6]:
v

21×8 Array{Float64,2}:
  0.0          0.0          0.0         …  0.0         0.0        
 -0.000538771  0.000482747  1.44942e-7     2.40118e-8  5.586e-5   
 -0.00107754   0.000965487  2.90211e-7     4.80619e-8  0.00011172 
 -0.00161629   0.00144821   4.36134e-7     7.21884e-8  0.00016758 
 -0.00215503   0.00193092   5.83041e-7     9.64297e-8  0.000223439
 -0.00269374   0.00241361   7.31256e-7  …  1.20824e-7  0.000279298
 -0.00323242   0.00289626   8.81109e-7     1.4541e-7   0.000335157
 -0.00377106   0.00337887   1.03293e-6     1.70224e-7  0.000391015
 -0.00430966   0.00386144   1.18704e-6     1.95307e-7  0.000446873
 -0.00484822   0.00434396   1.34376e-6     2.20696e-7  0.00050273 
 -0.00538672   0.00482642   1.50344e-6  …  2.46428e-7  0.000558586
 -0.00592516   0.00530882   1.66639e-6     2.72543e-7  0.000614442
 -0.00646353   0.00579115   1.83294e-6     2.99079e-7  0.000670296
 -0.00700183   0.0062734    2.00342e-6     3.26073e-7  0.000726149
 -0.00754005   0.00675557   2.17816e-6 

In [10]:
# Settign the time evolution

dt = 0.01

x0 = zeros(Natoms); # this is defined as an 2D array

for j = 1:Natoms
  x0[j] = (j-0.5)*Lat*Ndist+dx;
end

x1 = x0 + dt*[1, 0, 0, 0, 0, 0, 0, 0] # with velocity

(x, v, vdot) = time_evolution(velocity_verlet, x -> 10*forces(x), dt, 20, x0, x1)

initial conditions
  0.404047 seconds (555.64 k allocations: 102.731 MiB, 5.46% gc time)
running the loop
  0.480085 seconds (671.61 k allocations: 124.280 MiB, 4.18% gc time)
  0.491717 seconds (715.32 k allocations: 132.333 MiB, 3.64% gc time)
  0.485843 seconds (710.67 k allocations: 131.530 MiB, 3.70% gc time)
  0.497641 seconds (712.82 k allocations: 131.907 MiB, 3.66% gc time)
  0.522697 seconds (758.06 k allocations: 140.244 MiB, 3.48% gc time)
  0.523769 seconds (762.19 k allocations: 140.958 MiB, 4.07% gc time)
  0.521855 seconds (762.06 k allocations: 140.934 MiB, 3.44% gc time)
  0.520960 seconds (758.56 k allocations: 140.333 MiB, 4.14% gc time)
  0.523448 seconds (757.40 k allocations: 140.115 MiB, 3.51% gc time)
  0.547133 seconds (760.18 k allocations: 140.603 MiB, 4.01% gc time)
  0.567285 seconds (760.29 k allocations: 140.622 MiB, 4.05% gc time)
  0.557317 seconds (758.43 k allocations: 140.311 MiB, 3.93% gc time)
  0.538448 seconds (760.43 k allocations: 140.647 MiB,

([5.5 15.5 … 65.5 75.5; 5.51 15.5 … 65.5 75.5; … ; 5.68998 15.5 … 65.5 75.5; 5.69998 15.5 … 65.5 75.5], [1.0 0.0 … 0.0 0.0; 0.999999 3.49109e-7 … 4.56328e-9 3.31753e-7; … ; 0.999753 0.00013189 … 1.0618e-8 0.000115052; 0.999726 0.000146676 … 2.43297e-8 0.000127032], [-1.08667e-11 1.14146e-11 … -3.77979e-12 8.82502e-12; -0.000136425 6.98217e-5 … 9.12659e-7 6.63507e-5; … ; -0.00260906 0.00143653 … 1.38899e-6 0.00117035; -0.00274844 0.00152066 … 1.35335e-6 0.00122561])

In [11]:
x

21×8 Array{Float64,2}:
 5.5      15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.51     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.52     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.53     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.54     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.55     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.56     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.57     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.58     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.59     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.6      15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.61     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.62     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.63     15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.63999  15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.64999  15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.65999  15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.66999  15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.67999  15.5  25.5  35.5  45.5  55.5  65.5  75.5
 5.68998

In [12]:
v

21×8 Array{Float64,2}:
 1.0       0.0           0.0          …   0.0         0.0        
 0.999999  3.49109e-7   -6.73644e-9       4.56328e-9  3.31753e-7 
 0.999997  1.38529e-6   -1.28051e-8       5.89953e-9  1.33738e-6 
 0.999994  3.10833e-6   -1.60182e-8      -1.79856e-9  3.01647e-6 
 0.999989  5.53243e-6   -2.69283e-8      -1.10054e-8  5.35498e-6 
 0.999983  8.6737e-6    -3.26305e-8   …  -1.56102e-8  8.34756e-6 
 0.999975  1.25417e-5   -3.00847e-8      -1.5271e-8   1.19869e-5 
 0.999967  1.71374e-5   -2.76496e-8      -1.48179e-8  1.62645e-5 
 0.999956  2.24692e-5   -2.50945e-8      -1.45896e-8  2.11739e-5 
 0.999945  2.85464e-5   -2.22673e-8      -1.58023e-8  2.67096e-5 
 0.999932  3.53771e-5   -1.93511e-8   …  -2.11556e-8  3.28679e-5 
 0.999917  4.29664e-5   -1.67266e-8      -3.30421e-8  3.96471e-5 
 0.999902  5.13181e-5   -1.41541e-8      -4.96217e-8  4.70415e-5 
 0.999885  6.04396e-5   -1.07378e-8      -6.64514e-8  5.50404e-5 
 0.999866  7.03388e-5    3.56909e-10     -6.57654e-8 